## Michał Szczurek - laboratorium 4.

In [126]:
from enum import Enum
from spacy.tokenizer import Tokenizer
from spacy.language import Language
from spacy.vocab import Vocab
from random import random

### 1. Algorytm obliczania odległości edycyjnej

Funkcje i klasy pomocnicze:

In [127]:
class Operation(Enum):
    NONE = 0
    CHANGE = 1
    INSERT = 2
    DEL = 3 

In [128]:
class TableElement():   
    def __init__(self, cost=0, operation=None, parent=None):
        self.cost = cost
        self.operation = operation
        self.parent = parent
    def __repr__(self):
        return str(self.cost) 

In [129]:
def delta(x,y):
    if x==y:
        return 0
    return 1

Właściwy algorytm

Algorytm zwraca minimalną liczbę kroków oraz tablicę, na podstawie której można określić kolejność operacji (z tablicy tej korzysta algorytm z następnego punktu).

In [130]:
def edit_distance(source, dest, delta=delta):
    table = [[None for i in range(len(source)+1)] for j in range(len(dest)+1)]
    table[0][0] = TableElement()
    
    # prepare initial table
    for i in range(1,len(table[0])):
        table[0][i] = TableElement(i, Operation.DEL, table[0][i-1])
    for i in range(1,len(table)):
        table[i][0] = TableElement(i, Operation.INSERT, table[i-1][0])
    
    # fill the table
    for i in range(1, len(table)):
        for j in range(1,len(table[i])):
            min_element = table[i-1][j-1] 
            min_cost = table[i-1][j-1].cost
            operation = Operation.NONE # assume none operation is needed
            if dest[i-1] != source[j-1]: # check above assumption
                min_cost +=delta(dest[i-1], source[j-1])
                operation = Operation.CHANGE 
            if table[i][j-1].cost + 1 < min_cost: # check if deleting is not better
                min_element = table[i][j-1]
                min_cost = table[i][j-1].cost + 1
                operation = Operation.DEL
            if table[i-1][j].cost + 1 < min_cost: # check if adding is not better
                min_element = table[i-1][j]
                min_cost = table[i-1][j].cost + 1
                operation = Operation.INSERT
            table[i][j] = TableElement(min_cost, operation, min_element)
  
    return table[-1][-1].cost, table

### 2. Wizualizacja działania algorytmu

do wizualizaji możemy dać dodatkowy parametr table, oznaczający tablicę powstałą w algorytmie 1. Jeśli tego nie zrobimy, tablica zostanie automatycznie wyliczona korzystając ze wspomnianego algorytmu.

In [131]:
def visualize_operations(source, dest, table=None):
    if table is None:
        res, table = edit_distance(source, dest)
    element = table[-1][-1]
    operations = []
    while element.operation is not None:
        operations.append(element.operation)
        element = element.parent
    operations = operations[::-1]
    word = source
    x = 0
    y = 0
    for operation in operations:
        if operation == Operation.CHANGE:
            char = word[x]
            res = word[:x] + "*" + dest[y] + "*" + word[x+1:]
            word =  word[:x] + dest[y] + word[x+1:]
            print(res,f" > Zamiana {char} na {dest[y]}")
            x += 1
            y += 1
        elif operation == Operation.INSERT:
            res = word[:x] + "*" + dest[y] + "*" + word[x:]
            word = word[:x] + dest[y] + word[x:]
            print(res,f" > Dodanie {dest[y]}")
            x += 1
            y += 1
        elif operation == Operation.DEL:
            char = word[x]
            res = word[:x] + "*" + word[x+1:]
            word = word[:x] + word[x+1:]
            print(res,f" > Usunięcie {char}")
        else:
            x += 1
            y += 1

### 3. Wyniki działania algorytmów

In [132]:
res, table = edit_distance("los","kloc")
print(res)
visualize_operations("los", "kloc")

2
*k*los  > Dodanie k
klo*c*  > Zamiana s na c


In [133]:
res, table = edit_distance("Łódź", "Lodz")
print(res)
visualize_operations("Łódź", "Lodz")

3
*L*ódź  > Zamiana Ł na L
L*o*dź  > Zamiana ó na o
Lod*z*  > Zamiana ź na z


In [134]:
res, table = edit_distance("kwintesencja", "quintessence")
print(res)
visualize_operations("kwintesencja", "quintessence")

5
*q*wintesencja  > Zamiana k na q
q*u*intesencja  > Zamiana w na u
quinte*s*sencja  > Dodanie s
quintessenc*a  > Usunięcie j
quintessenc*e*  > Zamiana a na e


In [135]:
res, table = edit_distance("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")
print(res)
visualize_operations("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")

7
ATGA*G*TCTTACCGCCTCG  > Zamiana A na G
ATGAG*G*CTTACCGCCTCG  > Zamiana T na G
ATGAGGCT*C*TACCGCCTCG  > Dodanie C
ATGAGGCTCT*G*CCGCCTCG  > Zamiana A na G
ATGAGGCTCTG*G*CGCCTCG  > Zamiana C na G
ATGAGGCTCTGGC*C*CCTCG  > Zamiana G na C
ATGAGGCTCTGGCCCCT*G  > Usunięcie C


### Algorytm do obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów

Funkcja pomocnicza

In [136]:
def delta_lcs(x,y):
    if x==y:
        return 0
    return 2

Właściwy algorytm

In [137]:
def lcs(a,b):
    res, table = edit_distance(a, b, delta_lcs)
    element = table[-1][-1]
    i = len(a)
    res = []
    while element.operation is not None:
        if element.operation == Operation.NONE:
            res.append(a[i-1])
        if element.operation != Operation.INSERT:
            i -= 1
        element = element.parent
    return res[::-1]

### 5 i 6: Podział tekstu na tokeny i usunięcie 3% spośród nich

Dodałem opcję chroniącą przed usunięciem znako nowej linii. Ich usunięcie powodowało, że teksty momentami bywały "asynchroniczne", co widać w rezulatacie algorytmu diff. Wykonałem zadanie dla obu wersji.

In [166]:
def tokens_save(file_name, tokens):
    with open(file_name, 'w') as file:
        for token in text1:
            file.write(token.text_with_ws)

In [169]:
def tokens_load(file_name, tokens):
    with open(file_name, 'r') as file:
        text = file.read()
        tokenizer = Tokenizer(Language().vocab)
        tokens = tokenizer(text)
    return tokens

In [170]:
def del_tokens(tokens, percent=3, leave_lines=False):
    res = []
    for token in tokens:
        if leave_lines and str(token)[0] == "\n":
            res.append(token)
        elif random() > percent/100:
            res.append(token)
    return res

In [173]:
def prepare_files(file_name, leave_lines=False):
    with open(file_name, "r",encoding="utf8") as source:
        text = source.read()
        tokenizer = Tokenizer(Language().vocab)
        tokens = tokenizer(text)
        res1 = del_tokens(tokens, 3, leave_lines)
        tokens_save("res1.txt", res1)
        res2 = del_tokens(tokens, 3, leave_lines)
        tokens_save("res2.txt", res2)
    return res1, res2

### 7. Obliczenie długość najdłuższego podciągu wspólnych tokenów dla tekstów

In [189]:
text1, text2 = prepare_files("romeo-i-julia-700.txt")

In [190]:
print(len(text1))
print(len(text2))
print(len(lcs(text1, text2)))

2208
2192
2129


In [191]:
text1b, text2b = prepare_files("romeo-i-julia-700.txt", True)

In [192]:
print(len(text1b))
print(len(text2b))
print(len(lcs(text1b, text2b)))

2218
2217
2165


### 8. Implementacja diffa w oparciu o lcs

Przywrócenie znaków nowych linii

In [193]:
def restore_lines(tokens):
    res = []
    line = ""
    for token in tokens:
        if str(token)[0] == "\n": 
            res.append(line)
            line = ""
        else:
            line += token.text_with_ws
    return res

Właściwy algorytm

In [194]:
def diff(text1, text2):
    common = lcs(text1, text2)
    i = j = 0
    change = False
    for c in common:
        while text1[i] != c:
            change = True
            print("<", i, text1[i])
            i += 1
        while text2[j] != c:
            change = True
            print(">", j, text2[j])
            j += 1
        if change:
            print("========")
            change = False
        i += 1
        j += 1
        
    while i < len(text1):
        print("<", i, text1[i])
        i += 1
    while j < len(text2):
        print(">", j, text2[j])
        j += 1

### 9. Działanie algorytmu diff

Działanie prezentuję na dwóch rodzajach tekstów- takich z zachowanymi wszystkimi znakami nowej linii (b) i bez.

In [195]:
text1_resotred = restore_lines(text1)
text2_resotred = restore_lines(text2)

In [196]:
text1b_resotred = restore_lines(text1b)
text2b_resotred = restore_lines(text2b)

In [197]:
diff(text1_resotred ,text2_resotred )

< 5 * ESKALUS — książę panujący w 
> 5 * ESKALUS — książę panujący w Weronie
< 7 * MONTEKI, KAPULET naczelnicy dwóch domów nieprzyjaznych sobie
> 7 * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie
< 11 BENWOLIO — synowiec Montekiego
> 11 * BENWOLIO — synowiec Montekiego
< 13 * LAURENTY — ojciec franciszkanin
> 13 * LAURENTY — franciszkanin
< 20 * PARYSA
> 20 * PAŹ PARYSA
< 24 * KAPULET — małżonka Kapuleta
> 24 * PANI KAPULET — małżonka Kapuleta
< 27 * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa i inne osoby.
> 27 * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa i osoby.
< 30 Przełożył Jan Kasprowicz
> 30 Przełożył Kasprowicz
< 32 Tam, gdzie się rzecz ta rozgrywa, w Weronie,
< 33 Do nowej zbrodni pchają złości dawne,
> 32 Tam, gdzie się rzecz ta rozgrywa, w Weronie,Do nowej zbrodni pchają złości dawne,
< 40 I jak się ojców nienawiść nie zmienia,
< 41 A

In [198]:
diff(text1b_resotred ,text2b_resotred)

< 5 * ESKALUS — książę panujący w 
> 5 * ESKALUS — książę panujący w Weronie
< 10 * MERKUCJO — krewny księcia
> 10 * MERKUCJO — krewny 
< 19 * TRZECH 
> 19 * TRZECH MUZYKANTÓW
< 22 * DOWÓDCA WARTY
< 23 * PANI MONTEKI — małżonka 
> 22 * WARTY
> 23 * PANI MONTEKI — małżonka Montekiego
< 25 * JULIA — córka 
< 26 * MARTA mamka Julii
< 27 * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa inne osoby.
< 28 Rzecz odbywa się przez większą część sztuki w Weronie, przez część piątego aktu w Mantui.
> 25 * JULIA — córka Kapuletów
> 26 * MARTA — mamka Julii
> 27 * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa i inne osoby.
> 28 Rzecz odbywa się przez większą sztuki w Weronie, przez część piątego aktu w Mantui.
< 30 Przełożył Jan Kasprowicz
> 30 Jan Kasprowicz
< 49 Dalipan, Grzegorzu, będziem darli pierza.
> 49 Dalipan, Grzegorzu, nie będziem darli pierza.
< 54 
> 54 GRZEGORZ
< 59 al